# Скрипт для создания и заполнения БД

## Парсинг JSON и формирование sql скрипта

In [1]:
import json

In [2]:
DATABASE_OWNER = 'postgres' # Юзер БД (админ сервера)
ENCODING = 'UTF8' # Выбранная кодировка должна соответствовать кодировке сервера (на всякий случай)
LOCALE = 'ru_RU'

DATABASE_NAME = 'glemapanida'
SCHEMA_NAME = 'router'
TABLE_NAME = 'City'

In [3]:
create_db_q = f"""CREATE DATABASE {DATABASE_NAME} WITH
OWNER "{DATABASE_OWNER}"
ENCODING '{ENCODING}'
LC_COLLATE = '{LOCALE}.{ENCODING}'
LC_CTYPE = '{LOCALE}.{ENCODING}'
TEMPLATE template0;

\c {DATABASE_NAME};
set client_encoding TO '{ENCODING}';
"""

create_schema_table_q = f"""
CREATE SCHEMA {SCHEMA_NAME}
    CREATE TABLE {TABLE_NAME} (
        id SERIAL PRIMARY KEY,
        name TEXT NOT NULL,
        latitude NUMERIC(10, 7) NOT NULL,
        longitude NUMERIC(10, 7) NOT NULL
    );

CREATE INDEX name_idx ON {SCHEMA_NAME}.{TABLE_NAME} ((lower(name)));
"""

In [4]:
with open('cities.json', 'r', encoding=f'{ENCODING}') as f:
    city_data = json.load(f)

In [5]:
insert_into_table_q = f"""
"""

for city in city_data:
    city_name = list(city.keys())[0]
    city_lat = city[city_name]['latitude']
    city_lon = city[city_name]['longitude']
    
    insert_into_table_q += f"""INSERT INTO {SCHEMA_NAME}.{TABLE_NAME}(name, latitude, longitude) VALUES('{city_name}', {city_lat}, {city_lon});
"""

In [6]:
query_str = create_db_q + create_schema_table_q + insert_into_table_q

In [7]:
with open('create_db.sql', 'w', encoding=f'{ENCODING}') as f:
    f.write(query_str)

In [8]:
with open('create_db.sql', 'r', encoding=f'{ENCODING}') as f:
    print(f.read())

CREATE DATABASE glemapanida WITH
OWNER "postgres"
ENCODING 'UTF8'
LC_COLLATE = 'ru_RU.UTF8'
LC_CTYPE = 'ru_RU.UTF8'
TEMPLATE template0;

\c glemapanida;
set client_encoding TO 'UTF8';

CREATE SCHEMA router
    CREATE TABLE City (
        id SERIAL PRIMARY KEY,
        name TEXT NOT NULL,
        latitude NUMERIC(10, 7) NOT NULL,
        longitude NUMERIC(10, 7) NOT NULL
    );

CREATE INDEX name_idx ON router.City ((lower(name)));

INSERT INTO router.City(name, latitude, longitude) VALUES('Адыгейск', 44.8783715, 39.190172);
INSERT INTO router.City(name, latitude, longitude) VALUES('Майкоп', 44.6098268, 40.1006527);
INSERT INTO router.City(name, latitude, longitude) VALUES('Горно-Алтайск', 51.9582681, 85.9602957);
INSERT INTO router.City(name, latitude, longitude) VALUES('Алейск', 52.4920914, 82.7794145);
INSERT INTO router.City(name, latitude, longitude) VALUES('Барнаул', 53.3481145, 83.7798362);
INSERT INTO router.City(name, latitude, longitude) VALUES('Белокуриха', 51.9960534, 84.984

## Команды

__1. Создание и заполнение БД__: `psql -U postgres -q -b -f create_db.sql`\
__2. Проверка кодировки сервера__: `show server_encoding;`

__3. DB checks__
```SQL
select *
from router.city
where lower(name) like 'мо%';

select name, count(name) as dup_cnt
from router.city c 
group by name
having count(name) > 1;

select * from router.city where name = 'Ялта';

select count(*), (select count(*) from router.city)
from (
	select *
	from router.city
	group by id, name, latitude, longitude
) as dedup;
```